In [1]:
%pip install pyspark
%pip install delta-spark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 8.6 MB/s eta 0:00:0000:0100:02
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=66e562fe2191159d7a5ee5e4ed7b2b532abab02392d60b05748061ef6444a014
  Stored in directory: /home/jovyan/.cache/pip/wheels/95/13/41/f7f135ee114175605fb4f0a89e7389f3742aa6c1e1a5bcb657
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import DataFrame
from pyspark.sql.streaming import DataStreamWriter
from minio import Minio
from delta.tables import *
import os

In [4]:
spark = (
        SparkSession.builder
            .master("spark://spark-master-svc.lakehouse.svc.cluster.local:7077")
            .appName("raw_to_bronze")
            .config("spark.driver.host", "jupyter-thiago.lakehouse.svc.cluster.local")
            .config("spark.driver.port", 2222)
            ## Config Fields
            .config('spark.sql.debug.maxToStringFields', 5000)
            .config('spark.debug.maxToStringFields', 5000)
            ## Optimize
            .config("delta.autoOptimize.optimizeWrite", "true")
            .config("delta.autoOptimize.autoCompact", "true")
            ## Delta Table
            .config("spark.jars.packages", "io.delta:delta-core_2.12:2.3.0")
            .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
            # Resources
            .config("spark.executor.memory", "2g")
            #.config("spark.dynamicAllocation.enabled", "true")
            #.config("spark.submit.deployMode", "cluster")
            #.config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
            ## MinIO
            #.config("spark.hadoop.fs.s3a.endpoint", "http://tenant-lakehouse-hl.lakehouse.svc.cluster.local:9000")
            #.config('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
            .config("spark.hadoop.fs.s3a.proxy.host", "tenant-lakehouse-hl.lakehouse.svc.cluster.local")
            .config("spark.hadoop.fs.s3a.proxy.port", "9000")
            .config("spark.hadoop.fs.s3a.access.key", "mIQD7asW927IUQst")
            .config("spark.hadoop.fs.s3a.secret.key", "yBCgkZNdoG49SL8iowq4Uv2AmtDHd6G0")
            .config("spark.hadoop.fs.s3a.path.style.access", "true")
            .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
            .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
            #.config("spark.hadoop.fs.s3a.attempts.maximum", "1")
            .config("spark.hadoop.fs.s3a.connection.estabilish.timeout", "5000")
            #.config("spark.hadoop.fs.s3a.connection.timeout", "10000")
            ## Jars
            .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4,io.delta:delta-core_2.12:2.0.0")
            #.config("spark.jars", "/Users/thiagorente/Documentos/Python/lakehouse/spark/jars/hadoop-common-3.3.4.jar, \
            #                       /Users/thiagorente/Documentos/Python/lakehouse/spark/jars/hadoop-mapreduce-client-core-3.3.4.jar, \
            #                       /Users/thiagorente/Documentos/Python/lakehouse/spark/jars/hadoop-client-3.3.4.jar, \
            #                       /Users/thiagorente/Documentos/Python/lakehouse/spark/jars/aws-java-sdk-bundle-1.12.262.jar, \
            #                       /Users/thiagorente/Documentos/Python/lakehouse/spark/jars/hadoop-aws-3.3.4.jar")
            #.config("spark.driver.extraClassPath", \
            #                      "/Users/thomaz_rossito/driver_connection/jars/mysql-connector-j-8.0.32.jar")
            ## Hive SQL
            .enableHiveSupport()
            .getOrCreate()
    )

In [5]:
spark

In [6]:
log4jLogger = spark.sparkContext._jvm.org.apache.log4j
LOGGER = log4jLogger.LogManager.getLogger(__name__)
spark.sparkContext.setLogLevel("ALL")

In [7]:
spark.conf.set("spark.sql.shuffle.partitions", spark.sparkContext.defaultParallelism)

In [11]:
source_bucket = "raw"
prefix_bucket = "opencritic"
#source_path = f"s3a://{source_bucket}/{prefix_bucket}/2024-08-02_20:19:03.332882/data_opencritic.json"
source_path = f"s3a://{source_bucket}/{prefix_bucket}/"


In [9]:
games = spark.read.format('json').load(source_path)

#games = spark.read.json('/tmp/data.json')
games

DataFrame[date: string, list: array<array<struct<name:string,platforms:string,release_date:string,score:string>>>]

In [10]:
display(games.head())

Row(date='2024-08-02 20:19:03.332882', list=[[Row(name='Elden Ring: Shadow of the Erdtree', platforms='PC, PS5, PS4, XB1, XBXS', release_date='Jun 21', score='94')], [Row(name='Tsukihime -A piece of blue glass moon-', platforms='PS4, Switch', release_date='Jun 27', score='94')], [Row(name='Final Fantasy VII Rebirth', platforms='PS5', release_date='Feb 29', score='93')], [Row(name='Balatro', platforms='XBXS, PS5, Switch, PC, XB1, PS4', release_date='Feb 20', score='91')], [Row(name='Destiny 2: The Final Shape', platforms='PC, PS5, PS4, XB1, XBXS', release_date='Jun 4', score='91')], [Row(name='Hades II', platforms='PC', release_date='May 6', score='90')], [Row(name='Tekken 8', platforms='PC, PS5, XBXS', release_date='Jan 25', score='90')], [Row(name='Like A Dragon: Infinite Wealth', platforms='PC, PS5, PS4, XBXS, XB1', release_date='Jan 26', score='90')], [Row(name='The Last of Us Part II Remastered', platforms='PS5', release_date='Jan 19', score='89')], [Row(name='Animal Well', platfor

In [14]:
df_raw = spark \
        .readStream \
        .format("json") \
        .option("checkpointLocation", f"{source_path}/_checkpoint") \
        .schema(games.schema)\
        .load(source_path)

In [16]:
df_raw

DataFrame[date: string, list: array<array<struct<name:string,platforms:string,release_date:string,score:string>>>]

In [17]:
display(df_raw.head())

AnalysisException: Queries with streaming sources must be executed with writeStream.start();
FileSource[s3a://raw/opencritic/]

In [9]:
spark.stop()